<a href="https://colab.research.google.com/github/yingchengsun/Opinion-Spam-Detection/blob/master/Data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
from collections import OrderedDict
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import binascii
import pickle
import base64
import re
import sys
import os.path

yelp="https://www.yelp.com"

In [0]:
'''
transfer the original URL to 16 bit based coding address
'''
def transURL(s):
    #s="a b c"
    trans=''
    for x in s:
        code = ord(x)
        if (code>64 and code<91) or (code>96 and code<123) or (code>47 and code<58):
            trans += x
        else:
            #trans += "%" + x.encode("hex") #python 2.7
            x = x.encode('utf-8') #python 3
            trans += "%" + bytes.decode(base64.b16encode(x)) #python 3
    return trans

In [0]:
def download_images(reviewBody):
    pre_images=reviewBody.findAll('div',{"class":"photo-box photo-box--interactive is-loading"})
    for pre_image in pre_images:
        image = pre_image.find('img')
        image_url = image['data-async-src']
        image_name = image_url.split('/')[-2]
        urllib.urlretrieve(image_url,'pictures/'+'%s.jpg' % image_name)

        more =  reviewBody.find('li',{"class":"more-review-photos"})
        if more:
            print more.find('a')['href']

In [0]:
def extractReviews(url, num_of_reviews, biz_id):

    write_file_name = biz_id + ".json"
    with open("/content/drive/My Drive/Yelp/not_scraped.txt", "w+") as not_scraped:
      with open("/content/drive/My Drive/Yelp/Data/" + write_file_name, "w") as write_file:
        start = 0
        num_of_items_perpage = 20
        end = num_of_reviews
        while (start < end):
            url_ = url + '?start=' + str(start) + '&start='+ str(start)
            start += num_of_items_perpage
            #print (url_)
            page = urllib.request.urlopen(url_)
            soup = BeautifulSoup(page)
            
            #r= soup.find("script",{"type":"application/json","data-hypernova-key":"yelp_main__6685fde41a25f43c936f7d1d7c8536b9cd17b120__yelp_main__BizDetailsApp__dynamic"})
            #r= soup.find("script",{"type":"application/json","data-hypernova-key":"yelp_main__34ce8770cffec2357b054c3c9e43da6e193dbfd0__yelp_main__BizDetailsApp__dynamic"})
            #r= soup.find("script",{"type":"application/json","data-hypernova-key":"yelp_main__dbe4f988045111aa201e743b45b6bc83f96d44d4__yelp_main__BizDetailsApp__dynamic"})
            #r= soup.find("script",{"type":"application/json","data-hypernova-key":"yelp_main__5e83958d2ce784cbc6bb4524967f2af3e5536780__yelp_main__AdBizDetails__dynamic"})
            r= soup.find("script",{"type":"application/json","data-hypernova-key":"yelp_main__c0b64b766510882c5f527d24ec7f3ed5d144aa37__yelp_main__BizDetailsApp__dynamic"})
            
            n_r = soup.find("a", string=re.compile("not currently recommended"))
                                                                                  
            
            if not r:
              print (url_)
              print ("nothing") # make a collector
              not_scraped.write(biz_id)
              not_scraped.write('\n')
              return
            
            r_json = (r.text.lstrip('<!--').rstrip('-->'))
            data_item = json.loads(r_json, object_pairs_hook=OrderedDict)
            reviews = data_item['bizDetailsPageProps']['reviewFeedQueryProps']['reviews']
            for item in reviews:
              new_features_and_label = {}
              new_features_and_label['review_id'] = item['id']
              new_features_and_label['totalPhotos'] = item['totalPhotos']
              new_features_and_label['voterText'] = item['feedback']['voterText']
              new_features_and_label['label'] = 1
              json.dump(new_features_and_label, write_file)
              write_file.write('\n')
        
        
        n_r_URL = yelp + n_r['href']
        n_r_Number = n_r.text.split()[0]
        print (n_r_Number)
        
        start = 0
        num_of_items_perpage = 10
        end = int(n_r_Number)
        while (start < end):
            url_ = n_r_URL + '?not_recommended_start=' + str(start)
            print (url_)
            page = urllib.request.urlopen(url_)
            soup = BeautifulSoup(page)

            for item in soup.findAll('div',{"class":"review review--with-sidebar"}):
                new_features_and_label = {}
                new_features_and_label['review_id'] = item['data-review-id']
                
                photos = item.find("li", class_="photo-count responsive-small-display-inline-block")
                if photos != None:
                  new_features_and_label['totalPhotos'] = int(photos.b.text)
                else:
                  new_features_and_label['totalPhotos'] = 0

                new_features_and_label['reviews'] = item.find('div', class_="review-content").p.text
                if start > 0 and new_features_and_label['reviews'] == "This review has been removed for violating our Terms of Service":
                  continue
                new_features_and_label['label'] = 0
                #print (reviewBody.find('p').text)
                #ratings.append(reviewBody.div.meta['content'])
                
                json.dump(new_features_and_label, write_file)
                write_file.write('\n')
            start += num_of_items_perpage
    
   
      

In [0]:
'''
Collect "recommendated" and "not recommended" reviews by business ID 
#Main entrance
'''

with open('/content/drive/My Drive/Yelp/yelp_dataset/business.json', 'r') as biz_inf:
  i= 0
  for idx, line in enumerate(biz_inf):  
      if idx >= 100:
        break
      print (idx, line)
       
      '''
      read business from 'business.json'
      '''
      data_item = json.loads(line, object_pairs_hook=OrderedDict)
      biz_name = data_item['name']
      biz_id = data_item['business_id']

      if os.path.exists('/content/drive/My Drive/Yelp/Data/' +biz_id+'.json'):
        continue
      time.sleep(3)
      print (biz_name)
      biz_postal = data_item['postal_code']
      
      '''
      Search the business info and get the first one in the result list as the target
      '''
      search_url = yelp+"/search?find_desc="+transURL(biz_name)+"&find_loc="+transURL(biz_postal)
      print (search_url)
      search_result = urllib.request.urlopen(search_url)   
      soup = BeautifulSoup(search_result)    
      rankings = soup.findAll("p",{"class":"lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--black-regular__373c0__38bRH text-align--left__373c0__2pnx_ text-size--inherit__373c0__2gFQ3"})
      for item in rankings:
        if item.text[0:2]=="1.": #the first one in the result list
          biz_url = yelp + item.a['href']
          num_of_reviews = item.findNext("span",{"class":"lemon--span__373c0__3997G text__373c0__2pB8f reviewCount__373c0__2r4xT text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"}).text.split()[0]
          num_of_reviews = int(num_of_reviews)
          
      #print (biz_url)
      print (num_of_reviews)
      
      extractReviews(biz_url, num_of_reviews, biz_id)

                    

0 {"business_id":"1SWheh84yJXfytovILXOAQ","name":"Arizona Biltmore Golf Club","address":"2818 E Camino Acequia Drive","city":"Phoenix","state":"AZ","postal_code":"85016","latitude":33.5221425,"longitude":-112.0184807,"stars":3.0,"review_count":5,"is_open":0,"attributes":{"GoodForKids":"False"},"categories":"Golf, Active Life","hours":null}

1 {"business_id":"QXAEGFB4oINsVuTFxEYKFQ","name":"Emerald Chinese Restaurant","address":"30 Eglinton Avenue W","city":"Mississauga","state":"ON","postal_code":"L5R 3E7","latitude":43.6054989743,"longitude":-79.652288909,"stars":2.5,"review_count":128,"is_open":1,"attributes":{"RestaurantsReservations":"True","GoodForMeal":"{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","Caters":"True","NoiseLevel":"u'loud'","RestaurantsTableService":"True","RestaurantsTakeOut":"True","RestaurantsPriceRan

HTTPError: ignored

In [2]:
ids = ['mTbfremTz0Ev8UPzaoA-Zg',
'2zcUbKwnl2RIGnmtQh6gcA',
'6vB_8HIQGxsuJgHMzViOyg',
'1wnq0tSuIvzCp0rInC6t3A',
'yFOnkrLsGC0_-aEEAU0ghw',
'wLJhwxSY0DZFPx9133z1PA',
'bKhXOXuK4smzHJNO7BIJjg',
'Lu2sDbQBQ3rwKzHnLvGJ9w',
'fjrH7pwOtQE-q9IsSHWrFA',
'sD1rd8TjHbsEgUaXfvWnEA',
'07cHVHW7QZEyxI7XXufMWA',
'suC3ojSIGDtipxGwuCYzPQ',
'FYxFGv7H3_ep9ySk_dGRPA',
'Zz1Io406Gym-185LfSvgoA',
'B4nPQZosVDsZ2jQPsD9kNg',
'M0wwJYzp-YJLhrEGYnubMA']


with open('/content/drive/My Drive/Yelp/yelp_dataset/user.json', 'r') as user_inf:
  for id, line in enumerate(user_inf):
    print (line)
  

FileNotFoundError: ignored